In [1]:
import pandas as pd

# Leer Datos
df = pd.read_csv("./Data/df_final.csv")

# Contar cuántas veces ha postulado cada persona (numérico)
df['NRO_POSTULACION'] = df.groupby('APELLIDOS Y NOMBRES').cumcount() + 1

# Mostrar el resultado
print(df[['APELLIDOS Y NOMBRES', 'NRO_POSTULACION']].head())


            APELLIDOS Y NOMBRES  NRO_POSTULACION
0      ABAD GRANDA, ANDRE FAHET                1
1       ABAD NEYRA, ANDREA TAIS                1
2  ABAL YACSAYAURI, SOFIA BELEN                1
3    ABANTO ARAUJO, PERCY ANGEL                1
4  ABANTO AYALA, LARISSA TAMARA                1


In [2]:
import numpy as np

# Función para generar una edad basada en NRO_POSTULACION
def generar_edad(nro_postulacion):
    min_edad = 16 + (nro_postulacion - 1)
    max_edad = 20 + (nro_postulacion - 1)
    return np.random.randint(min_edad, max_edad + 1)

# Aplicar la función a cada fila
df['EDAD'] = df['NRO_POSTULACION'].apply(generar_edad)

# Ver resultado
print(df[['APELLIDOS Y NOMBRES', 'NRO_POSTULACION', 'EDAD']].head())

            APELLIDOS Y NOMBRES  NRO_POSTULACION  EDAD
0      ABAD GRANDA, ANDRE FAHET                1    17
1       ABAD NEYRA, ANDREA TAIS                1    18
2  ABAL YACSAYAURI, SOFIA BELEN                1    17
3    ABANTO ARAUJO, PERCY ANGEL                1    17
4  ABANTO AYALA, LARISSA TAMARA                1    18


In [ ]:
import pandas as pd
from unidecode import unidecode

# Leer el archivo names_ine.tsv con nombres de columnas personalizados
names_df = pd.read_csv("./Data/data/names_ine.tsv", sep="\t", header=None, names=["name", "frequency", "is_male"])

# Asegurar que los nombres en names_df estén en minúsculas
names_df["name"] = names_df["name"].str.strip().str.lower()

# Convertir la columna 'is_male' en texto (M/F)
def convertir_genero(valor):
    if valor == 1:
        return "M"
    elif valor == 0:
        return "F"
    else:
        return "ND"
    
# Extraer primer nombre
def extraer_nombre(nombre_completo):
    try:
        if "," in nombre_completo:
            partes = nombre_completo.split(",")
            if len(partes) < 2:
                return None
            nombres = partes[1].strip()
        else:
            nombres = nombre_completo.strip()
        primer_nombre = nombres.split()[0].lower()
        return unidecode(primer_nombre)
    except:
        return None
    

# Función para predecir el sexo a partir del primer nombre
def predecir_genero(nombre):
    if not nombre:
        return "ND"
    fila = names_df[names_df["name"] == nombre]
    if not fila.empty:
        # Asegurarse de que el género se tome como una decisión binaria
        probabilidad = fila.iloc[0]["is_male"]
        # Si la probabilidad es mayor que 0.5, asignamos 'M'; si no, 'F'
        return "M" if probabilidad > 0.5 else "F"
    return "ND"

In [10]:
# Extraer primer nombre
df['PRIMER_NOMBRE'] = df['APELLIDOS Y NOMBRES'].apply(extraer_nombre)

# Predecir sexo
df['SEXO'] = df['PRIMER_NOMBRE'].apply(predecir_genero)


In [11]:

# Asignar 'M' o 'F' aleatoriamente a los que tienen 'ND'
nd_mask = df["SEXO"] == "ND"
df.loc[nd_mask, "SEXO"] = np.random.choice(["M", "F"], size=nd_mask.sum())

# Verificar resultado
print(df[["APELLIDOS Y NOMBRES", "PRIMER_NOMBRE", "SEXO"]].head())

            APELLIDOS Y NOMBRES PRIMER_NOMBRE SEXO
0      ABAD GRANDA, ANDRE FAHET         andre    M
1       ABAD NEYRA, ANDREA TAIS        andrea    F
2  ABAL YACSAYAURI, SOFIA BELEN         sofia    F
3    ABANTO ARAUJO, PERCY ANGEL         percy    M
4  ABANTO AYALA, LARISSA TAMARA       larissa    F


In [12]:
# Asignar aleatoriamente 'Particular' y 'Estatal' con las probabilidades 55% y 45%
df['COLEGIO_PROCENDENCIA'] = np.random.choice(
    ['Particular', 'Estatal'], 
    size=len(df), 
    p=[0.55, 0.45]  # Probabilidades para 'Particular' y 'Estatal'
)

# Verificar el resultado
print(df[['APELLIDOS Y NOMBRES', 'COLEGIO_PROCENDENCIA']].head())

            APELLIDOS Y NOMBRES COLEGIO_PROCENDENCIA
0      ABAD GRANDA, ANDRE FAHET              Estatal
1       ABAD NEYRA, ANDREA TAIS           Particular
2  ABAL YACSAYAURI, SOFIA BELEN           Particular
3    ABANTO ARAUJO, PERCY ANGEL              Estatal
4  ABANTO AYALA, LARISSA TAMARA           Particular


In [ ]:
# Eliminar la columna 'PRIMER_NOMBRE'
df.drop(columns=['PRIMER_NOMBRE'], inplace=True)



In [16]:
# Agrupar por AÑO, PERIODO, y CARRERA, y contar el número total de postulantes en cada grupo
postulantes = df.groupby(['AÑO', 'PERIODO', 'CARRERA']).size().reset_index(name='POSTULANTES')

# Unir este conteo de postulantes con el DataFrame original (df)
df = df.merge(postulantes, on=['AÑO', 'PERIODO', 'CARRERA'], how='left')

# Verificar el resultado
print(df[['APELLIDOS Y NOMBRES', 'AÑO', 'PERIODO', 'CARRERA', 'ESTATUS', 'POSTULANTES']].head())


            APELLIDOS Y NOMBRES   AÑO PERIODO          CARRERA     ESTATUS  \
0      ABAD GRANDA, ANDRE FAHET  2023       I  MEDICINA HUMANA  NO INGRESO   
1       ABAD NEYRA, ANDREA TAIS  2023       I  MEDICINA HUMANA  NO INGRESO   
2  ABAL YACSAYAURI, SOFIA BELEN  2023       I  MEDICINA HUMANA  NO INGRESO   
3    ABANTO ARAUJO, PERCY ANGEL  2023       I  MEDICINA HUMANA  NO INGRESO   
4  ABANTO AYALA, LARISSA TAMARA  2023       I  MEDICINA HUMANA  NO INGRESO   

   POSTULANTES  
0         3896  
1         3896  
2         3896  
3         3896  
4         3896  


In [32]:
# Filtrar solo los registros con ESTATUS == 'INGRESO'
ingreso_df = df[df['ESTATUS'] == 'INGRESO']

# Agrupar por AÑO, PERIODO, y CARRERA, y contar cuántos 'INGRESO' hay en cada grupo
vacantes = ingreso_df.groupby(['AÑO', 'PERIODO', 'CARRERA']).size().reset_index(name='VACANTES')

# Unir este conteo de vacantes con el DataFrame original (df)
df = df.merge(vacantes, on=['AÑO', 'PERIODO', 'CARRERA'], how='left')

# Verificar el resultado
print(df[['APELLIDOS Y NOMBRES', 'AÑO', 'PERIODO', 'CARRERA', 'ESTATUS', 'VACANTES']].head())


            APELLIDOS Y NOMBRES   AÑO PERIODO          CARRERA     ESTATUS  \
0      ABAD GRANDA, ANDRE FAHET  2023       I  MEDICINA HUMANA  NO INGRESO   
1       ABAD NEYRA, ANDREA TAIS  2023       I  MEDICINA HUMANA  NO INGRESO   
2  ABAL YACSAYAURI, SOFIA BELEN  2023       I  MEDICINA HUMANA  NO INGRESO   
3    ABANTO ARAUJO, PERCY ANGEL  2023       I  MEDICINA HUMANA  NO INGRESO   
4  ABANTO AYALA, LARISSA TAMARA  2023       I  MEDICINA HUMANA  NO INGRESO   

   VACANTES  
0        55  
1        55  
2        55  
3        55  
4        55  


In [30]:
# Eliminar la columna 'PRIMER_NOMBRE'
df.drop(columns=['PRIMER_NOMBRE'], inplace=True)

In [34]:
# Guardar el DataFrame actualizado en un archivo CSV
df.to_csv('./Data/df_final_con_colegio.csv', index=False)

In [33]:
df.head()

,CODIGO,APELLIDOS Y NOMBRES,CARRERA,PUNTAJE,PUESTO,ESTATUS,MODALIDAD,SEDE,AÑO,PERIODO,NRO_POSTULACION,EDAD,SEXO,COLEGIO_PROCENDENCIA,POSTULANTES,VACANTES
0,108704,"ABAD GRANDA, ANDRE FAHET",MEDICINA HUMANA,1007.125,,NO INGRESO,EDUCACIÓN BÁSICA REGULAR (EBR) Y EDUCACIÓN BÁS...,LIMA,2023,I,1,17,M,Estatal,3896,55
1,159787,"ABAD NEYRA, ANDREA TAIS",MEDICINA HUMANA,480.375,,NO INGRESO,EDUCACIÓN BÁSICA REGULAR (EBR) Y EDUCACIÓN BÁS...,LIMA,2023,I,1,18,F,Particular,3896,55
2,100330,"ABAL YACSAYAURI, SOFIA BELEN",MEDICINA HUMANA,504.875,,NO INGRESO,EDUCACIÓN BÁSICA REGULAR (EBR) Y EDUCACIÓN BÁS...,LIMA,2023,I,1,17,F,Particular,3896,55
3,121362,"ABANTO ARAUJO, PERCY ANGEL",MEDICINA HUMANA,733.625,,NO INGRESO,EDUCACIÓN BÁSICA REGULAR (EBR) Y EDUCACIÓN BÁS...,LIMA,2023,I,1,17,M,Estatal,3896,55
4,139925,"ABANTO AYALA, LARISSA TAMARA",MEDICINA HUMANA,733.625,,NO INGRESO,EDUCACIÓN BÁSICA REGULAR (EBR) Y EDUCACIÓN BÁS...,LIMA,2023,I,1,18,F,Particular,3896,55
